In [13]:
import copy
import numpy as np

In [14]:
def optimality_check(c):
    if [rdcd_cost for rdcd_cost in c if rdcd_cost > 1e-8]:
        return True
    else:
        return False

In [20]:
def pivoting(c,A,b,z):
    entering = (np.where(c == max(c)))[0][0]

    ratio = {row_idx:rhs/lhs[entering] for row_idx,(lhs,rhs) in enumerate(zip(A,b)) if lhs[entering] >= 0}

    min_ratio_idx = min(ratio, key=ratio.get)
    mul_val = float(1/A[min_ratio_idx][entering])

    criteria_lhs = copy.copy(A[min_ratio_idx]*mul_val)
    criteria_rhs = copy.copy(b[min_ratio_idx]*mul_val)
    
    for row_idx,(lhs,rhs) in enumerate(zip(A,b)):
        pivot_val = float(lhs[entering])
        if row_idx != min_ratio_idx:
#             if lhs[entering] > 0:
            A[row_idx] = lhs - (criteria_lhs*pivot_val)
            b[row_idx] = rhs - (criteria_rhs*pivot_val)
#             else:
#                 A[row_idx] = lhs + (criteria_lhs*pivot_val)
#                 b[row_idx] = rhs + (criteria_rhs*pivot_val)
        else:
            A[row_idx] = criteria_lhs
            b[row_idx] = criteria_rhs
    
    odj_pivot_val = c[entering]
#     if odj_pivot_val > 0:
    c -= criteria_lhs*odj_pivot_val
    z -= criteria_rhs*odj_pivot_val
#     else:
#         c += criteria_lhs*odj_pivot_val
#         z += criteria_rhs*odj_pivot_val
        
    return c,A,b,z

In [21]:
def unbounded_check(col):
    if [num for num in col if num < 0]:
        return True
    else:
        return False

In [17]:
def print_tableau(step, c,A,b,z):
    print(f'===== step {step} =====')
    print(c, z)
    for lhs,rhs in zip(A,b):
        print(lhs, rhs)
    print()

In [18]:
c = np.array([2,1,1,0,0,0], dtype=np.float)
A = np.array([
    [3,1,1,1,0,0],
    [1,-1,2,0,1,0],
    [1,1,-1,0,0,1]
], dtype=np.float)
b = np.array([60,10,20], dtype=np.float)
z = 0

In [18]:
step = 1
while optimality_check(c):
    c,A,b,z = pivoting(c,A,b,z)
    
    if unbounded_check(row[2] for row in A):
        print('unbounded!')
        break
    
    print_tableau(step, c,A,b,z)
    
    step += 1

===== step 1 =====
[ 0.  3. -3.  0. -2.  0.] -20.0
[ 0.  4. -5.  1. -3.  0.] 30.0
[ 1. -1.  2.  0.  1.  0.] 10.0
[ 0.  2. -3.  0. -1.  1.] 10.0

===== step 2 =====
[ 0.   0.   1.5  0.  -0.5 -1.5] -35.0
[ 0.  0.  1.  1. -1. -2.] 10.0
[1.  0.  0.5 0.  0.5 0.5] 15.0
[ 0.   1.  -1.5  0.  -0.5  0.5] 5.0

===== step 3 =====
[ 0.   0.   0.  -1.5  1.   1.5] -50.0
[ 0.  0.  1.  1. -1. -2.] 10.0
[ 1.   0.   0.  -0.5  1.   1.5] 10.0
[ 0.   1.   0.   1.5 -2.  -2.5] 20.0

===== step 4 =====
[-1.  0.  0. -1.  0.  0.] -60.0
[1.33333333 0.         1.         0.33333333 0.33333333 0.        ] 23.333333333333332
[ 0.66666667  0.          0.         -0.33333333  0.66666667  1.        ] 6.666666666666666
[ 1.66666667  1.          0.          0.66666667 -0.33333333  0.        ] 36.666666666666664



# Make Problem & Standard Form

In [129]:
class Problem:
    def __init__(self, prob_type=None):
        self.variables = []
        self.constraints = []
        self.prob_type = prob_type
        
    def add_variable(self, name=None, obj_coef=None, lb=0, ub=float('inf')):
        variable = Variable(name, obj_coef, lb, ub)
        self.variables.append(variable)
        
    def add_constraint(self, name=None, lhs=None, rhs=None, ineq_type=None):
        constraint = Constraint(name, lhs, rhs, ineq_type)
        self.constraints.append(constraint)

    def set_prob_type(self, prob_type):
        self.prob_type = prob_type
        
        
class Variable:
    def __init__(self, name=None, obj_coef=None, lb=0, ub=float('inf')):
        self.name = name
        self.obj_coef = obj_coef
        self.lb = lb
        self.ub = ub
        
    def set_name(self,name):
        self.name = name
        
    def set_obj_coef(self,obj_coef):
        self.obj_coef = obj_coef
        
    def set_lb(self, lb):
        self.lb = lb
        
    def set_ub(self, ub):
        self.lb = lb
        
        
class Constraint:
    def __init__(self, name=None, lhs=None, rhs=None, ineq_type=None):
        self.name = name
        self.lhs = lhs
        self.rhs = rhs
        self.ineq_type = ineq_type
        
    def set_lhs_coef(self, lhs):
        self.lhs = lhs
        
    def set_rhs(self, rhs):
        self.rhs = rhs
        
    def set_ineq_type(self, ineq_type):
        self.ineq_type = ineq_type
        
        

# Example

Max 2x1 + x2 + x3  

s.t 3x1 + x2 + x3 <= 60  
&nbsp;&nbsp;&nbsp;&nbsp; x1 - x2 + 2x3 <= 10  
&nbsp;&nbsp;&nbsp;&nbsp; x1 + x2 - x3 <= 20  
&nbsp;&nbsp;&nbsp;&nbsp; x1,x2,x3 >= 0  

In [135]:
c = np.array([2,1,1])
A = np.array([
    [3,1,1],
    [1,-1,2],
    [1,1,-1]
    ])
b = np.array([60,-10,20])

types = np.array(['L','L','L'])

num_var = len(c)
num_const = len(A)

problem = Problem()

for i in range(num_var):
    problem.add_variable(name = f'x{i}', obj_coef = c[i])

for i in range(num_const):
    problem.add_constraint(name = f'c{i}', lhs = A[i], rhs = b[i], ineq_type=types[i])


In [136]:
for i in problem.variables:
    print(i.obj_coef, i.name)
    


2 x0
1 x1
1 x2


In [137]:
for i in problem.constraints:
    print(i.lhs, i.ineq_type, i.rhs)
    
    

[3 1 1] L 60
[ 1 -1  2] L -10
[ 1  1 -1] L 20


In [146]:
# def make_standard_form(problem):
    # rhs 양수
    # variable 0보다 크거나 같다.
    # 모든 제약식의 부호가 =
    
    # 일단 변수들 조정시켜주자. x <= 0 -> -x역할을 하는 y생성, x is free -> s2 - s1으로
    
std_variables = []
std_constraints = []
std_ineq_types = []

for v in problem.variables:
    if v.lb == -float('inf') and v.ub == 0:
        alt_variable = Variable(name=f'{v.name}_y', obj_coef=-v.obj_coef, lb=0, ub=float('inf'))
        std_variables.append(alt_variable)

    elif v.lb == -float('inf') and v.ub == float('inf'):
        plus_variable = Variable(name=f'{v.name}_plus', obj_coef=v.obj_coef, lb=0, ub=float('inf'))
        minus_variable = Variable(name=f'{v.name}_minus', obj_coef=-v.obj_coef, lb=0, ub=float('inf'))
        std_variables.append(plus_variable)
        std_variables.append(minus_variable)

    else:
        variable = Variable(name=f'{v.name}', obj_coef=v.obj_coef, lb=v.lb, ub=v.ub)
        std_variables.append(variable)

# 제약에서 rhs 음수이면 부호바꿔준다.

for c in problem.constraints:
    if c.rhs < 0:
        alt_c = Constraint(name = f'c{i}', lhs = -c.lhs, rhs = -c.rhs, ineq_type='R' if t == 'L' else 'L')
        std_constraints.append(alt_c)
    else:
        original_c = Constraint(name = f'c{i}', lhs = c.lhs, rhs = c.rhs, ineq_type=c.ineq_type)
        std_constraints.append(original_c)
        
# 제약에서 부호 L이면 슬랙, R이면 surplus, artificial 추가, E면 artificial추가



            

In [150]:
for c in std_constraints:
    print(c.lhs, c.ineq_type ,c.rhs)

[3 1 1] L 60
[-1  1 -2] R 10
[ 1  1 -1] L 20


In [145]:
t = 'L'
'R' if t == 'L' else 'L' 

'R'

In [134]:
for i in std_variables:
    print(i.name)
    
    
    

x0
x1
x2


In [127]:
a = np.array([[1,2,3],[4,5,6]])

b = np.array([[7,8,9]])


In [128]:
np.concatenate((a,b))

array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]])

In [119]:
b

array([1, 2, 3, 4, 5, 6])

In [139]:
a

array([[1, 2, 3],
       [4, 5, 6]])

In [140]:
-a

array([[-1, -2, -3],
       [-4, -5, -6]])

In [141]:
c

In [142]:
c.rhs

20